# What is unit testing?

Unit testing is the practice of creating 'tests' that allow you to automatically check that individual parts of your code are working as expected.

Unit testing should be done at the level of individual functions or methods so that if your code fails a test, you can quickly identify where the error is coming from. It can still be useful to create tests that check that your code works when you try and run everything together, but this should be done alongside unit testing, and not instead of unit testing! 

# Why do we do unit testing?

Unit testing allows you to:
* Identify if your code is broken
* Identify where and in what way your code is broken
* Save you time by doing this automatically instead of requiring you to manually run lines of code (this is particularly useful in collaborative projects where you might want to test code that somebody else has written before you merge a change into your main branch)

Taking unit tests can take time (especially when you've only just started using them), but the more unit tests you write for your project, the faster you'll be able to fix problems as the project gets bigger, and the more confident you (and others) will be that your code actually does what you think it does.

If you've never done unit testing before, you don't need to paralyse yourself by making a unit test for everything! It's great to have lots of unit tests, but even only having some unit tests is better than having no unit tests! Particularly for Data Science projects, you might not necessarily know what output to expect from a piece of code you write, making it difficult to write a unit test, just try your best to write tests wherever you can!

# How do we do unit testing?

There are lots of ways to write to write unit tests. The one I will focus on here is `pytest`, but other tools are available and you should feel free to use whatever tools suit you best!

## Setting up

We'll need to import a few packages to use `pytest`, namely `pytest` itself!

So that I can show the output of tests within this notebook, I'm also using a package called [`ipytest`](https://github.com/chmp/ipytest/blob/main/Example.ipynb), but most of the time you'll want to simply run pytest from the terminal and won't require this package.

`hypothesis` is a package that includes some more advanced testing features. We'll come back to look at `hypothesis` later.

In [1]:
import pytest
from hypothesis import given, strategies as st

import ipytest
ipytest.autoconfig()

## Writing a test

Let's define a simple function that adds one number to another number...

In [2]:
def adds_x(num, x):
    return(num + x)

print(adds_x(1, 2))

3


To test this function with a unit test, we need to write a unit test function...

In [ ]:
def test_adds_x():
    result = adds_x(3, 1)
    expect = 4
    assert result == expect

There are three key parts to this test:
* **The name of the test function** - It is important that the test name begins with `test_`! This allows pytest to identify the tests associated with your package and automatically run them all for you. The function name should also make it clear what you are testing.
* **The code that tests your function** - Obviously, if we want to test a function, we'll have to use that function within the test and compare the result we get to the result we expect and make sure they're the same.
* **The `assert` statement** - This line determines whether your test passess or fails. If the statement is true, then the test passes. If the statement is false, then the test fails.

## Running a test

Most of the time, you'll be running `pytest` by using the `pytest` command from the terminal within your repo (we'll go through how this works later). However, in this example, I will run the test from within the notebook so you can see the test output in the same place!

In [3]:
%%ipytest

def test_adds_x():
    result = adds_x(3, 1)
    expect = 4
    assert result == expect

.                                                                                            [100%]
======================================== warnings summary =========================================
..\..\..\..\..\python36\lib\site-packages\_pytest\config\__init__.py:1126
  c:\python36\lib\site-packages\_pytest\config\__init__.py:1126: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: hypothesis
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
1 passed, 1 warning in 0.02s


As we can see, our test passed! This is because the assert statement was true (i.e. the result produced by our function was the result that we were expecting)!

Running the test will also display warning by defaults, even if your test passes.

But what if our test fails?...

In [4]:
%%ipytest

def test_adds_x_fail():
    result = adds_x(3, 1) + 1
    expect = 4
    assert result == expect

F                                                                                            [100%]
============================================ FAILURES =============================================
________________________________________ test_adds_x_fail _________________________________________

    def test_adds_x_fail():
        result = adds_x(3, 1) + 1
        expect = 4
>       assert result == expect
E       assert 5 == 4

<ipython-input-4-8c5fc7a60feb>:4: AssertionError
======================================== warnings summary =========================================
..\..\..\..\..\python36\lib\site-packages\_pytest\config\__init__.py:1126
  c:\python36\lib\site-packages\_pytest\config\__init__.py:1126: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: hypothesis
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
===================================== short test summary info ============

Here our test failed (because the assert statement was false). This tells us which test failed, but if we want, we can also add a string after the assert statement that will tell you what went wrong when the test fails (great if you've got a complicated test case and you might forget what it tests for)...

In [5]:
%%ipytest

def test_adds_x_fail():
    result = adds_x(3, 1) + 1
    expect = 4
    assert result == expect, f"adds_x should return 4 when given 3 and 1, but instead returned {result}"

F                                                                                            [100%]
============================================ FAILURES =============================================
________________________________________ test_adds_x_fail _________________________________________

    def test_adds_x_fail():
        result = adds_x(3, 1) + 1
        expect = 4
>       assert result == expect, f"adds_x should return 4 when given 3 and 1, but instead returned {result}"
E       AssertionError: adds_x should return 4 when given 3 and 1, but instead returned 5
E       assert 5 == 4

<ipython-input-5-f44bbbd21ef7>:4: AssertionError
======================================== warnings summary =========================================
..\..\..\..\..\python36\lib\site-packages\_pytest\config\__init__.py:1126
  c:\python36\lib\site-packages\_pytest\config\__init__.py:1126: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: hypothesis
    self._mark_plugin

## Automating tests (a bit)

In the previous example, the test inputs and outputs were hard-coded in. But we can use a feature of `pytest` to define a series of inputs. This allows you to automatically run a series of tests without having to write a new test function to test a different input.

We do this by using a `parametrize` decorator (the line of code that starts with an '@') in front of the test function that tells the test to use a defined series of inputs. You have to pass this decorator two things:
* A string that contains the names of the parameters to automate the input for
* A list of tuples, where each tuple comtains the values you want to pass in one iteration of the test (the values need to be given in the same order that you named the parameters in)

In [6]:
%%ipytest

@pytest.mark.parametrize("num, x", [(3, 1), (0, -1), (-5, 2)])
def test_adds_x(num, x):
    result = adds_x(num, x)
    expect = num+1
    assert result == expect, f"adds_x should return {expect} when given {num} and {x}, but instead returned {result}"

.FF                                                                                          [100%]
============================================ FAILURES =============================================
________________________________________ test_adds_x[0--1] ________________________________________

num = 0, x = -1

    @pytest.mark.parametrize("num, x", [(3, 1), (0, -1), (-5, 2)])
    def test_adds_x(num, x):
        result = adds_x(num, x)
        expect = num+1
>       assert result == expect, f"adds_x should return {expect} when given {num} and {x}, but instead returned {result}"
E       AssertionError: adds_x should return 1 when given 0 and -1, but instead returned -1
E       assert -1 == 1

<ipython-input-6-d7958550b5a1>:5: AssertionError
________________________________________ test_adds_x[-5-2] ________________________________________

num = -5, x = 2

    @pytest.mark.parametrize("num, x", [(3, 1), (0, -1), (-5, 2)])
    def test_adds_x(num, x):
        result = adds_x(num, x

You can see that instead of just running one test, you actually just ran three! If you look at the first line, you can see a green '.' and two red 'F's, this means that the 1st test passed while the 2nd and 3rd tests failed.

If it makes sense for your test, you can use a mixture of hard-coding and parametrizing...

In [7]:
%%ipytest

@pytest.mark.parametrize("num", [(-1), (0), (1), (100000000)])
def test_adds_one(num):
    x = 1
    result = adds_x(num, x)
    expect = num + x
    assert result == expect, f"adds_x should return {expect} when given {num} and {x}, but instead returned {result}"

....                                                                                         [100%]
======================================== warnings summary =========================================
..\..\..\..\..\python36\lib\site-packages\_pytest\config\__init__.py:1126
  c:\python36\lib\site-packages\_pytest\config\__init__.py:1126: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: hypothesis
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
4 passed, 1 warning in 0.05s


# How can I make unit testing less of a slog?

Even using parametrization can take a lot of time! You need to think of and type out as many edge cases as you can think of, so you're likely to only be feasibly able to test a small subset of possible cases.

We can use the `hypothesis` package in Python to simplify this for us! `hypothesis` uses property-based testing, which just means it can perform tests on a particular type of input rather than a set of inputs you explicitly define.

We do this by using a `given` decorator in front of the test function that tells the test what sort of inputs to test. Unlike `pytest`'s `parametrize` decorator, you don't need to name the parameters you're passing, the arguments are simply passed in order.

Let's look at a test function that uses the `given` decorator...

In [8]:
%%ipytest

@given(st.integers())
def test_adds_one(num):
    x = 1
    result = adds_x(num, x)
    expect = num + x
    assert result == expect, f"adds_x should return {expect} when given {num} and {x}, but instead returned {result}"

.                                                                                            [100%]
======================================== warnings summary =========================================
..\..\..\..\..\python36\lib\site-packages\_pytest\config\__init__.py:1126
  c:\python36\lib\site-packages\_pytest\config\__init__.py:1126: PytestAssertRewriteWarning: Module already imported so cannot be rewritten: hypothesis
    self._mark_plugins_for_rewrite(hook)

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
1 passed, 1 warning in 0.15s


This appears to have run a single test (only one green '.'), but if we take a look at what's actually happening...

In [9]:
%%ipytest -s

@given(st.integers())
def test_adds_one(num):
    print(f"Trying {num}...")
    x = 1
    result = adds_x(num, x)
    expect = num + x
    assert result == expect, f"adds_x should return {expect} when given {num} and {x}, but instead returned {result}"

Trying 0...
Trying 0...
Trying 21108...
Trying 0...
Trying -13061...
Trying 0...
Trying 344233119867605228...
Trying 0...
Trying 0...
Trying 0...
Trying -16026...
Trying -62...
Trying 26095...
Trying -101...
Trying -13299...
Trying -51...
Trying -97356405197373059265829707354439238195...
Trying -9917...
Trying 384...
Trying -4579...
Trying -111...
Trying 29...
Trying 2024...
Trying 33...
Trying -2180053...
Trying -1054235964690920472...
Trying -27444...
Trying 8418...
Trying -32...
Trying -2829...
Trying -25348...
Trying 24945...
Trying 97...
Trying -8840857843637675326594082683253669381...
Trying 13...
Trying -2731...
Trying -10...
Trying 99847871871469761951544532910454698335...
Trying -359670906...
Trying -9798...
Trying 38...
Trying -72...
Trying 1880091742...
Trying 83...
Trying -5737...
Trying -29508...
Trying 115...
Trying 1710447166...
Trying 74...
Trying -817859607...
Trying -168290035611739574479419243320659295672...
Trying 4214...
Trying 19126...
Trying -31494...
Trying 103.

Note that you can include print statements in your tests. By default, this won't display in your `pytest` output, but if you run `pytest` with the `-s` flag, any print statements will be displayed as your tests run!

This 'one' test is actually running 100 different tests of auto-generated numbers that cover a large range of cases (0, negative numbers, large numbers, large negative numbers...), but condesing the result into one output to prevent clutering your test results.

If the test fails on any input, it will show you the output that the test failed on...

In [10]:
%%ipytest

@given(st.integers())
def test_adds_one_fail_on_zero(num):
    x = 1
    result = adds_x(num, x)
    if num == 0:
        result = 0
    expect = num + x
    assert result == expect, f"adds_x should return {expect} when given {num} and {x}, but instead returned {result}"

F                                                                                            [100%]
============================================ FAILURES =============================================
___________________________________ test_adds_one_fail_on_zero ____________________________________

    @given(st.integers())
>   def test_adds_one_fail_on_zero(num):

<ipython-input-10-f087c17378cc>:2: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

num = 0

    @given(st.integers())
    def test_adds_one_fail_on_zero(num):
        x = 1
        result = adds_x(num, x)
        if num == 0:
            result = 0
        expect = num + x
>       assert result == expect, f"adds_x should return {expect} when given {num} and {x}, but instead returned {result}"
E       AssertionError: adds_x should return 1 when given 0 and 1, but instead returned 0
E       assert 0 == 1

<ipython-input-10-f087c17378cc>:8: AssertionError
------------------

`hypothesis` is clever, and if your test fails on a particular input value, it will make sure to include the same value in future test runs. It also has all sorts of cool features to cover a whole range of tests, so I'd recommend reading through [the docs](https://hypothesis.readthedocs.io/en/latest/)!